<a href="https://colab.research.google.com/github/smabb/p/blob/master/Lesson6%20Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--NAVIGATION-->


<a href="https://colab.research.google.com/github/smabb/p/blob/master/Lesson6 Pandas.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>



# Pandas

In the NumPy section we dealt with some arrays, whose columns had each a special meaning. For example, the column number 0 could contain values interpreted as years, and column 1 could contain a month, and so on. It is possible to handle the data this way, but in can be hard to remember, which column number corresponds to which variable. Especially, if you later remove some column from the array, then the numbering of the remaining columns changes. One solution to this is to give a descriptive name to each column. These column names stay fixed and attached to their corresponding columns, even if we remove some of the columns. In addition, the rows can be given names as well, these are called *indices* in Pandas.

The [Pandas](http://pandas.pydata.org/) library is built on top of the NumPy library, and it provides a special kind of two dimensional data structure called `DataFrame`. The `DataFrame` allows to give names to the columns, so that one can access a column using its name in place of the index of the column.

First we will quickly go through a few examples to see what is possible with Pandas. You may need to check some details from the Pandas [documentation](http://pandas.pydata.org/pandas-docs/stable/) in order to complete the exercises. We start by doing some standard imports:

In [0]:
import pandas as pd    # This is the standard way of importing the Pandas library
import numpy as np

Let's import some weather data that is in text form in a csv (Commma Separated Values) file. The following call will fetch the data from the internet and convert it to a DataFrame:

In [0]:
wh = pd.read_csv("https://raw.githubusercontent.com/smabb/p/master/data/temp.csv")
wh.head()   # The head method prints the first 5 rows

,Year,m,d,Time,Time zone,Precipitation amount (mm),Snow depth (cm),Air temperature (degC)
0,2017,1,1,00:00,UTC,-1.0,-1.0,0.6
1,2017,1,2,00:00,UTC,4.4,-1.0,-3.9
2,2017,1,3,00:00,UTC,6.6,7.0,-6.5
3,2017,1,4,00:00,UTC,-1.0,13.0,-12.8
4,2017,1,5,00:00,UTC,-1.0,10.0,-17.8


We see that the DataFrame contains eight columns, three of which are actual measured variables. Now we can refer to a column by its name:

In [0]:
wh["Snow depth (cm)"].head()     # Using the tab key can help enter long column names

There are several summary statistic methods that operate on a column or on all the columns. The next example computes the mean of the temperatures over all rows of the DataFrame:

In [0]:
wh["Air temperature (degC)"].mean()    # Mean temperature

We can drop some columns from the DataFrame with the `drop` method:

In [0]:
wh.drop("Time zone", axis=1).head()    # Return a copy with one column removed, the original DataFrame stays intact

In [0]:
wh.head()     # Original DataFrame is unchanged

In case you want to modify the original DataFrame, you can either assign the result to the original DataFrame, or use the `inplace` parameter of the `drop` method. Many of the modifying methods of the DataFrame have the `inplace` parameter.

Addition of a new column works like adding a new key-value pair to a dictionary:

In [0]:
wh["Rainy"] = wh["Precipitation amount (mm)"] > 5
wh.head()

In the next sections we will systematically go through the DataFrame and its one-dimensional version: *Series*.

## Creation and indexing of series

One can turn any one-dimensional iterable into a Series, which is a one-dimensional data structure:

In [0]:
s=pd.Series([1, 4, 5, 2, 5, 2])
s

The data type of the elements in this Series is `int64`, integers representable in 64 bits. 

We can also attach a name to this series:

In [0]:
s.name = "Grades"
s

The common attributes of the series are the `name`, `dtype`, and `size`:

In [0]:
print(f"Name: {s.name}, dtype: {s.dtype}, size: {s.size}")

In addition to the values of the series, also the row indices were printed. All the accessing methods from NumPy arrays also work for the Series: indexing, slicing, masking and fancy indexing. 

In [0]:
s[1]

In [0]:
s2=s[[0,5]]                    # Fancy indexing
print(s2)

In [0]:
t=s[-2:]                    # Slicing
t

Note that the indices stick to the corresponding values, they are not renumbered!

In [0]:
t[4]                        # t[0] would give an error

The values as a NumPy array are accessible via the `values` attribute:

In [0]:
s2.values

And the indices are available through the `index` attribute:

In [0]:
s2.index

The index is not simply a NumPy array, but a data structure that allows fast access to the elements. The indices need not be integers, as the next example shows:

In [0]:
s3=pd.Series([1, 4, 5, 2, 5, 2], index=list("abcdef"))
s3

In [0]:
s3.index

In [0]:
s3["b"]

<div class="alert alert-warning">
Note a special case here: if the indices are not integers, then the last index of the slice is included in the result. This is contrary to slicing with integers!
</div>

In [0]:
s3["b":"e"]

It is still possible to access the series using NumPy style *implicit integer indices*:

In [0]:
s3[1]

This can be confusing though. Consider the following series:

In [0]:
s4 = pd.Series(["Jack", "Jones", "James"], index=[1,2,3])
s4

What do you think `s4[1]` will print? For this ambiguity Pandas offers attributes `loc` and `iloc`. The attributes `loc` always uses the explicit index, while the attribute `iloc` always uses the implicit integer index:

In [0]:
print(s4.loc[1])
print(s4.iloc[1])

# Pandas (continues)

In [0]:
import pandas as pd
import numpy as np

## Creation of dataframes

The DataFrame is essentially a two dimensional object, and it can be created in three different ways:

* out of a two dimensional NumPy array
* out of given columns
* out of given rows

### Creating DataFrames from a NumPy array

In the following example a DataFrame with 2 rows and 3 column is created. The row and column indices are given explicitly.

In [0]:
df=pd.DataFrame(np.random.randn(2,3), columns=["First", "Second", "Third"], index=["a", "b"])
df

Note that now both the rows and columns can be accessed using the special `Index` object:

In [0]:
df.index                            # These are the "row names"

In [0]:
df.columns                          # These are the "column names"

If either `columns` or `index` argument is left out, then an implicit integer index will be used:

In [0]:
df2=pd.DataFrame(np.random.randn(2,3), index=["a", "b"])
df2

Now the column index is an object similar to Python's builtin `range` type:

In [0]:
df2.columns

### Creating DataFrames from columns

A column can be specified as a list, an NumPy array, or a Pandas' Series. The names of the columns can be given either with the `columns` parameter, or if Series objects are used, then the `name` attribute of each Series is used as the column name.

In [0]:
s1 = pd.Series([1,2,3])
s1

In [0]:
s2 = pd.Series([4,5,6], name="b")
s2

Give the column name explicitly:

In [0]:
pd.DataFrame(s1, columns=["a"])

Use the `name` attribute of Series s2 as the column name:

In [0]:
pd.DataFrame(s2)

If using multiple columns, then they must be given as the dictionary, whose keys give the column names and values are the actual column content.

In [0]:
pd.DataFrame({"a": s1, "b": s2})

### Creating DataFrames from rows

We can give a list of rows as a parameter to the DataFrame constructor. Each row is given as a dict, list, Series, or NumPy array. If we want to give names for the columns, then either the rows must be dictionaries, where the key is the column name and the values are the elements of the DataFrame on that row and column, or else the column names must be given explicitly. An example of this:

In [0]:
df=pd.DataFrame([{"Wage" : 1000, "Name" : "Jack", "Age" : 21}, {"Wage" : 1500, "Name" : "John", "Age" : 29}])
df

Or:

In [0]:
df = pd.DataFrame([[1000, "Jack", 21], [1500, "John", 29]], columns=["Wage", "Name", "Age"])
df

<div class="alert alert-warning">Note that the order of columns is not always the same order as they were in the parameter list. In this case you can use the `columns` parameter to specify the exact order.


In the earlier case, however, where we created DataFrames from a dictionary of columns, the order of columns should be the same as in the parameter dictionary in the recent versions of Python and Pandas.
</div>

In the sense of information content the order of columns should not matter, but sometimes you want to specify a certain order to make the Frame more readable, or to make it obey some semantic meaning of column order.

## Accessing columns and rows of a dataframe

Even though DataFrames are basically just two dimensional arrays, the way to access their elements is different from NumPy arrays. There are a couple of complications, which we will go through in this section.

Firstly, the bracket notation `[]` does not allow the use of an index pair to access a single element of the DataFrame. Instead only one dimension can be specified.

Well, does this dimension specify the rows of the DataFrame, like NumPy arrays if only one index is given, or does it specify the columns of the DataFrame?

It depends!

If an integer is used, then it specifies a column of the DataFrame in the case the **explicit** indices for the column contain that integer. In any other case an error will result. For example, with the above DataFrame, the following indexing will not work, because the explicit column index consist of the column names "Name" and "Wage" which are not integers.

In [0]:
try:
    df[0]
except KeyError:
    import sys
    print("Key error", file=sys.stderr)

The following will however work.

In [0]:
df["Wage"]

As does the fancy indexing:

In [0]:
df[["Wage", "Name"]]

If one indexes with a slice or a boolean mask, then the **rows** are referred to. Examples of these:

In [0]:
df[0:1]                           # slice

In [0]:
df[df.Wage > 1200]               # boolean mask

If some of the above calls return a Series object, then you can chain the bracket calls to get a single value from the DataFrame:

In [0]:
df["Wage"][1]                    # Note order of dimensions

But there is a better way to achieve this, which we will see in the next section.

## Alternative indexing and data selection

If the explanation in the previous section sounded confusing or ambiguous, or if you didn't understand a thing, you don't have to worry.

There is another way to index Pandas DataFrames, which

* allows use of index pairs to access a single element
* has the same order of dimensions as NumPy: first index specifies rows, second columns
* is not ambiguous about implicit or explicit indices

Pandas DataFrames have attributes `loc` and `iloc` that have the above qualities.
You can use `loc` and `iloc` attributes and forget everything about the previous section. Or you can use these attributes
and sometimes use the methods from the previous section as shortcuts if you understand them well.

The difference between `loc` and `iloc` attributes is that the former uses explicit indices and the latter uses the implicit integer indices. Examples of use:

In [0]:
df.loc[1, "Wage"]

In [0]:
df.iloc[-1,-1]             # Right lower corner of the DataFrame

In [0]:
df.loc[1, ["Name", "Wage"]]

With `iloc` everything works like with NumPy arrays: indexing, slicing, fancy indexing, masking and their combinations. With `loc` it is the same but now the names in the explicit indices are used for specifying rows and columns. Make sure your understand why the above examples work as they do!

## Summary statistics

The summary statistic methods work in a similar way as their counter parts in NumPy. By default, the aggregation is done over columns.

In [0]:
wh = pd.read_csv("https://raw.githubusercontent.com/smabb/p/master/data/temp.csv")

In [0]:
wh2 = wh.drop(["Year", "m", "d"], axis=1)  # taking averages over these is not very interesting
wh2.mean()

The `describe` method of the `DataFrame` object gives different summary statistics for each (numeric) column. The result is a DataFrame. This method gives a good overview of the data, and is typically used in the exploratory data analysis phase.

In [0]:
wh.describe()

## Missing data

You may have noticed something strange in the output of the `describe` method. First, the minimum value in both precipitation and snow depth fields is -1. The special value -1 means that on that day there was absolutely no snow or rain, whereas the value 0 might indicate that the value was close to zero. Secondly, the snow depth column has count 358, whereas the other columns have count 365, one measurement/value for each day of the year. How is this possible? Every field in a DataFrame should have the same number of rows. Let's use the `unique` method of the Series object to find out, which different values are used in this column:

In [0]:
wh["Snow depth (cm)"].unique()

The `float` type allows a special value `nan` (Not A Number), in addition to normal floating point numbers. This value can represent the result from an illegal operation. For example, the operation 0/0 can either cause an exception to occur or just silently produce a `nan`. In Pandas `nan` can be used to represent a missing value. In the weather DataFrame the `nan` value tells us that the measurement from that day is not available, possibly due to a broken measuring instrument or some other problem.

Note that only float types allow the `nan` value (in Python, NumPy or Pandas). So, if we try to create an integer series with missing values, its dtype gets promoted to `float`:

In [0]:
pd.Series([1,3,2])

In [0]:
pd.Series([1,3,2, np.nan])

For non-numeric types the special value `None` is used to denote a missing value, and the dtype is promoted to `object`.

In [0]:
pd.Series(["jack", "joe", None])

Pandas excludes the missing values from the summary statistics, like we saw in the previous section. Pandas also provides some functions to handle missing values.

The missing values can be located with the `isnull` method:

In [0]:
wh.isnull()      # returns a boolean mask DataFrame

This is not very useful as we cannot directly use the mask to index the DataFrame. We can, however, combine it with the `any` method to find out all the rows that contain at least one missing value:

In [0]:
wh[wh.isnull().any(axis=1)]

The `notnull` method works conversively to the `isnull` method.

The `dropna` method of a DataFrame drops columns or rows that contain missing values from the DataFrame, depending on the `axis` parameter.

In [0]:
wh.dropna().shape   # Default axis is 0

In [0]:
wh.dropna(axis=1).shape # Drops the columns containing missing values

The `how` and `thresh` parameters of the `dropna` method allow one to specify how many values need to be missing in order for the row/column to be dropped.

The `fillna` method allows to fill the missing values with some constant or interpolated values. The `method` parameter can be:

* `None`: use the given positional parameter as the constant to fill missing values with
* `ffill`: use the previous value to fill the current value
* `bfill`: use the next value to fill the current value

For example, for the weather data we could use forward fill

In [0]:
wh = wh.fillna(method='ffill')
wh[wh.isnull().any(axis=1)]

The `interpolate` method, which we will not cover here, offers more elaborate ways to interpolate the missing values from their neighbouring non-missing values.

## Converting columns from one type to another

There are several ways of converting a column to another type. For converting single columns (a Series) one can use the `pd.to_numeric` function or the `map` method. For converting several columns in one go one can use the `astype` method. We will give a few examples of use of these methods/functions. For more details, look from the Pandas documentation.

In [0]:
pd.Series(["1","2"]).map(int)                           # str -> int

In [0]:
pd.Series([1,2]).map(str)                               # int -> str

In [0]:
pd.to_numeric(pd.Series([1,1.0]), downcast="integer")   # object -> int

In [0]:
pd.to_numeric(pd.Series([1,"a"]), errors="coerce")      # conversion error produces Nan

In [0]:
pd.Series([1,2]).astype(str)                            # works for a single series

In [0]:
df = pd.DataFrame({"a": [1,2,3], "b" : [4,5,6], "c" : [7,8,9]})
print(df.dtypes)
print(df)

In [0]:
df.astype(float)                       # Convert all columns

In [0]:
df2 = df.astype({"b" : float, "c" : str})    # different types for columns
print(df2.dtypes)
print(df2)

## String processing

If the elements in a column are strings, then the vectorized versions of Python's string processing methods are available. These are accessed through the `str` attribute of a Series or a DataFrame. For example, to capitalize all the strings of a Series, we can use the `str.capitalize` method:

In [0]:
names = pd.Series(["donald", "theresa", "angela", "vladimir"])
names.str.capitalize()

One can find all the available methods by pressing the tab key after the text `names.str.` in a Python prompt. Try it in below cell!

In [0]:
#names.str.

We can split a column or Series into several columns using the `split` method. For example:

In [0]:
full_names = pd.Series(["Donald Trump", "Theresa May", "Angela Merkel", "Vladimir Putin"])
full_names.str.split()

This is not exactly what we wanted: now each element is a list. We need to use the `expand` parameter to split into columns:

In [0]:
full_names.str.split(expand=True)

## Additional information

We covered subsetting of DataFrames with the indexers `[]`, `.loc[]`, and `.iloc[]` quite concisely.
For a more verbose explanation, look at the [tutorials at Dunder Data](https://medium.com/dunder-data/pandas-tutorials/home). Especially, the problems with chained indexing operators (like `df["a"][1]`) are explained well there (tutorial 4), which we did not cover at all. As a rule of thumb: one should avoid chained indexing combined with assignment! See [Pandas documentation](http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#why-does-assignment-fail-when-using-chained-indexing).

# Pandas (continues)

In [0]:
import pandas as pd
import numpy as np

## Catenating datasets

We already saw in the NumPy section how we can catenate arrays along an axis: `axis=0` catenates vertically and `axis=1` catenates horizontally, and so on. With the DataFrames of Pandas it works similarly except that the row indices and the column names require extra attention. Also note a slight difference in the name: `np.concatenate` but `pd.concat`.

Let's start by considering catenation along the axis 0, that is, vertical catenation. We will first make a helper function to easily create DataFrames for testing.

In [0]:
def makedf(cols, ind):
    data = {c : [str(c) + str(i) for i in ind] for c in cols}
    return pd.DataFrame(data, ind)

Next we will create some example DataFrames:

In [0]:
a=makedf("AB", [0,1])
a

In [0]:
b=makedf("AB", [2,3])
b

In [0]:
c=makedf("CD", [0,1])
c

In [0]:
d=makedf("BC", [2,3])
d

In the following simple case, the `concat` function works exactly as we expect it would:

In [0]:
pd.concat([a,b])   # The default axis is 0

The next, however, will create duplicate indices:

In [0]:
r=pd.concat([a,a])
r

In [0]:
r.loc[0,"A"]

This is not usually what we want! There are three solutions to this. Firstly, deny creation of duplicated indices by giving the `verify_integrity` parameter to the `concat` function:

In [0]:
try:
    pd.concat([a,a], verify_integrity=True)
except ValueError as e:
    import sys
    print(e, file=sys.stderr)

Secondly, we can ask for automatic renumbering of rows:

In [0]:
pd.concat([a,a], ignore_index=True)

Thirdly, we can ask for *hierarchical indexing*. The indices can contain multiple levels, but on this course we don't consider hierarchical indices in detail. Hierarchical indices can make a two dimensional array to work like higher dimensional array.

In [0]:
r2=pd.concat([a,a], keys=['first', 'second'])
r2

In [0]:
r2["A"]["first"][0]

Everything works similarly, when we want to catenate horizontally:

In [0]:
pd.concat([a,c], axis=1)

We have so far assumed that when concatenating vertically the columns of both DataFrames are the same, and when joining horizontally the indices are the same. This is, however, not required:

In [0]:
pd.concat([a,d], sort=False)    # sort option is used to silence a deprecation message

It expanded the non-existing cases with `NaN`s. This method is called an *outer join*, which forms the union of columns in the two DataFrames. The alternative is *inner join*, which forms the intersection of columns:

In [0]:
pd.concat([a,d], join="inner")

## Merging dataframes

Merging combines two DataFrames based on some common field.

Let's recall the earlier DataFrame about wages and ages of persons:

In [0]:
df = pd.DataFrame([[1000, "Jack", 21], [1500, "John", 29]], columns=["Wage", "Name", "Age"])
df

Now, create a new DataFrame with the occupations of persons:

In [0]:
df2 = pd.DataFrame({"Name" : ["John", "Jack"], "Occupation": ["Plumber", "Carpenter"]})
df2

The following function call will merge the two DataFrames on their common field, and, importantly, will keep the indices *aligned*. What this means is that even though the names are listed in different order in the two frames, the merge will still give correct result.

In [0]:
pd.merge(df, df2)

This was an example of a simple one-to-one merge, where the keys in the `Name` columns had 1-to-1 correspondence. Sometimes not all the keys appear in both DataFrames:

In [0]:
df3 = pd.concat([df2, pd.DataFrame({ "Name" : ["James"], "Occupation":["Painter"]})], ignore_index=True)
df3

In [0]:
pd.merge(df, df3)                # By default an inner join is computed

In [0]:
pd.merge(df, df3, how="outer")   # Outer join

Also, many-to-one and many-to-many relationships can occur in merges:

In [0]:
books = pd.DataFrame({"Title" : ["War and Peace", "Good Omens", "Good Omens"] , 
                      "Author" : ["Tolstoi", "Terry Pratchett", "Neil Gaiman"]})
books

In [0]:
collections = pd.DataFrame([["Oodi", "War and Peace"],
                           ["Oodi", "Good Omens"],
                           ["Pasila", "Good Omens"],
                           ["Kallio", "War and Peace"]], columns=["Library", "Title"])
collections

All combinations with matching keys (`Title`) are created:

In [0]:
libraries_with_books_by = pd.merge(books, collections)
libraries_with_books_by

## Aggregates and groupings

Let us use again the weather dataset. First, we make the column names a bit more uniform and concise. For example the columns `Year`, `m`, and `d` are not uniformly named.

We can easily change the column names with the `rename` method of the DataFrame. Note that we cannot directly change the index `wh.columns` as it is immutable.

In [0]:
wh = pd.read_csv("https://raw.githubusercontent.com/smabb/p/master/data/temp.csv")

In [0]:
wh3 = wh.rename(columns={"m": "Month", "d": "Day", "Precipitation amount (mm)" : "Precipitation", 
                         "Snow depth (cm)" : "Snow", "Air temperature (degC)" : "Temperature"})
wh3.head()

Pandas has an operation that splits a DataFrame into groups, performs some operation on each of the groups, and then combines the result from each group into a resulting DataFrame. This split-apply-combine functionality is really flexible and powerful operation. In Pandas you start by calling the `groupby` method, which splits the DataFrame into groups. In the following example the rows that contain measurements from the same month belong to the same group:

In [0]:
groups = wh3.groupby("Month")
groups

Nothing happened yet, but the `groupby` object knows how the division into groups is done. This is called a lazy operation. We can query the number of groups in the `groupby` object:

In [0]:
len(groups)

We can iterate through all the groups:

In [0]:
for key, group in groups:
    print(key, len(group))

In [0]:
groups.get_group(2)                 # Group with index two is February

The `groupby` object functions a bit like a DataFrame, so some operations which are allowed for DataFrames are also allowed for the `groupby` object. For example, we can get a subset of columns:

In [0]:
groups["Temperature"]

For each DataFrame corresponding to a group the Temperature column was chosen. Still nothing was shown, because we haven't applied any operation on the groups.

The common methods also include the aggregation methods. Let's try to apply the `mean` aggregation:

In [0]:
groups["Temperature"].mean()

Now what happened was that after the mean aggregation was performed on each group, the results were automatically combined into a resulting DataFrame. Let's try some other aggregation:

In [0]:
groups["Precipitation"].sum()

Ok, the -1.0 values in the Precipitation field are causing trouble here, let's convert them to zeros:

In [0]:
wh4 = wh3.copy()
wh4.loc[wh4.Precipitation == -1, "Precipitation"] = 0
wh4.loc[wh4.Snow == -1, "Snow"] = 0
wh4.head()

In [0]:
wh4.groupby("Month")["Precipitation"].sum()

### Other ways to operate on groups

The aggregations are not the only possible operations on groups. The other possibilities are filtering, transformation, and application.

In **filtering** some of the groups can be filtered out.

In [0]:
def myfilter(df):                                     # The filter function must return a boolean value
    return df["Precipitation"].sum() >= 150

wh4.groupby("Month").filter(myfilter)                 # Filter out months with total precipitation less that 150 mm

In a **transformation** each group's DataFrame is manipulated in a way that retains its shape. An example of centering values, so that the deviations from the monthly means are shown:

In [0]:
pd.concat([wh4.iloc[:, 0:3], 
           wh4.groupby("Month")[["Precipitation", "Snow", "Temperature"]].transform(lambda x : x - x.mean())], 
          axis=1)

The **apply** method is very generic and only requires that for each group's DataFrame the given function returns a DataFrame, Series, or a scalar. In the following example, we sort within each group by the temperature:

In [0]:
wh4.groupby("Month").apply(lambda df : df.sort_values("Temperature"))

## Time series

If a measurement is made at certain points in time, the resulting values with their measurement times is called a time series. In Pandas a Series whose index consists of dates/times is a time series.

Let's make a copy of the DataFrame that we can mess with:

In [0]:
wh2 = wh3.copy()
wh2.columns

The column names `Year`, `Month`, and `Day` are now in appropriate form for the `to_datetime` function. It can convert these fields into a timestamp series, which we will add to the DataFrame.

In [0]:
wh2["Date"] = pd.to_datetime(wh2[["Year", "Month", "Day"]])
wh2.head()

We can now drop the useless fields:

In [0]:
wh2=wh2.drop(columns=["Year", "Month", "Day"])
wh2.head()

The following method call will set the Date field as the index of the DataFrame.

In [0]:
wh2 = wh2.set_index("Date")
wh2.head()

We can now easily get a set of rows using date slices:

In [0]:
wh2["2017-01-15":"2017-02-03"]

By using the `date_range` function even more complicated sets can be formed. The following gets all the Mondays of July:

In [0]:
r=pd.date_range("2017-07-01", "2017-07-31", freq="w-mon")
r

In [0]:
wh2.index.difference(r)

In [0]:
wh2.loc[r,:]

The following finds all the business days (Monday to Friday) of July:

In [0]:
pd.date_range("2017-07-01", "2017-07-31", freq="b")

We can get a general idea about the `Temperature` column by plotting it. Note how the index time series is shown nicely on the x-axis.

In [0]:
%matplotlib inline
wh2["Temperature"].plot();

The graph looks a bit messy at this level of detail. We can smooth it by taking averages over a sliding window of length 30 days:

In [0]:
rolling = wh2.Temperature.rolling(30, center=True)
rolling

In [0]:
data = pd.DataFrame({"Temperature" : wh2.Temperature, "Rolling mean" : rolling.mean()})
data.plot();

## Additional information

[Pandas cheat sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf) Summary of most important Pandas' functions and methods.

Read the article [Tidy Data](https://www.jstatsoft.org/article/view/v059i10/v59i10.pdf). The article uses the statistical software R as an example, but the ideas are relevant in general. Pandas operations maintain data in the tidy format.

Pandas handles only one dimensional data (Series) and two dimensional data (DataFrame). While you can use [hierarchical indices](http://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#hierarchical-indexing-multiindex) to simulate higher dimensional arrays, you should use the [xarray](http://xarray.pydata.org/en/stable/index.html) library, if you need proper higher-dimensional arrays with labels. It is basically a cross between NumPy and Pandas.

